In [ ]:
# tsv = [sample key, image url, class name, verification label, h-dimensional feature delimited by ','].

In [4]:
# emb_file = feat_dir + 'extracted_emb.txt'
# img_path_file = feat_dir + 'extracted_imgpaths.txt'

# save_train_tsv_file = feat_dir + 'cleannet_train.tsv'
# save_val_tsv_file = feat_dir + 'cleannet_val.tsv'
# save_all_tsv_file = feat_dir + 'cleannet_all.tsv'
# save_class_names_file = feat_dir + 'cleannet_classnames.txt'
# np.savetxt(save_class_names_file, tsv['class_name'].unique(), fmt='%s')

import numpy as np
import pandas as pd
print("ok")

ok


In [7]:
## FOOD

def create_verified_tsv(type_name):
    
#     extracted_img_paths_file = "../trained_models/food101n_20e/extracted_imgpaths.txt"
#     extracted_emb_file = '../trained_models/food101n_20e/extracted_emb.txt'
#     strip_field = '../datasets/Food-101N_release/images/'
#     if type_name == 'verified_train':
#         v_list = "../datasets/Food-101N_release/meta/verified_train.tsv"
#         save_tsv_file = "../trained_models/food101n_20e/cleannet_train.tsv"
#         vdf_colnames = ['image_url', 'verification_label']
#     elif type_name == 'verified_val':
#         v_list = "../datasets/Food-101N_release/meta/verified_val.tsv"
#         save_tsv_file = "../trained_models/food101n_20e/cleannet_val.tsv"
#         vdf_colnames = ['image_url', 'verification_label']
#     elif type_name == 'all':
#         save_tsv_file = "../trained_models/food101n_20e/cleannet_all.tsv"
        
    extracted_img_paths_file = "../trained_models/round1_food101n_20e/extracted_imgpaths.txt"
    extracted_emb_file = '../trained_models/round1_food101n_20e/extracted_emb.txt'
    strip_field = '../datasets/Food-101N_release/images/'
    if type_name == 'verified_train':
        v_list = "../datasets/Food-101N_release/meta/verified_train.tsv"
        save_tsv_file = "../trained_models/round1_food101n_20e/cleannet_train.tsv"
        vdf_colnames = ['image_url', 'verification_label']
    elif type_name == 'verified_val':
        v_list = "../datasets/Food-101N_release/meta/verified_val.tsv"
        save_tsv_file = "../trained_models/round1_food101n_20e/cleannet_val.tsv"
        vdf_colnames = ['image_url', 'verification_label']
    elif type_name == 'all':
        save_tsv_file = "../trained_models/round1_food101n_20e/cleannet_all.tsv"
        
    extracted_img_paths = pd.read_csv(extracted_img_paths_file, header=None, index_col=None)
    extracted_img_paths=extracted_img_paths[0].apply(lambda s: s.split(strip_field)[1])
    # print("extracted_img_paths", extracted_img_paths.head())
    
    if type_name in ['verified_train', 'verified_val']:
        vdf = pd.read_csv(v_list, '\t')
        vdf.columns = vdf_colnames
        ind = extracted_img_paths.index[extracted_img_paths.isin(vdf['image_url'])].tolist()
        print("verified df", vdf.head())
        print("ind", ind[0:10])
    
        # sample_key	image_url	class_name	verification_label	feature
        t = pd.DataFrame()
        t['sample_key'] = ind
        t['image_url'] = np.array(extracted_img_paths.iloc[ind])
        t['class_name'] = t['image_url'].map(lambda s: s.split('/')[0])
        t = t.merge(vdf)
        # iter_csv = pd.read_csv(extracted_emb_file, iterator=True, header=None, chunksize=1000)
        # emb = pd.concat([chunk[chunk.index.isin(ind)] for chunk in iter_csv])
        a = np.arange(len(extracted_img_paths))
        set_ind = set(ind)
        skip = [i for i in a if i not in set_ind]
        print("skip", skip[0:10])
        emb_df = pd.read_csv(extracted_emb_file, skiprows=skip, header=None)
        t['feature'] = emb_df[0].apply(lambda e: ",".join(map(str, e.split())))

        t.to_csv(save_tsv_file, sep='\t', index=False, header=None)
        print("tsv created", t.head())
    elif type_name == 'all':
        # iter_csv = pd.read_csv(extracted_emb_file, iterator=True, header=None, chunksize=1000)
        # emb = pd.concat([chunk[chunk.index.isin(ind)] for chunk in iter_csv])
        t = pd.DataFrame()
        t['sample_key'] = np.arange(len(extracted_img_paths))
        t['image_url'] = extracted_img_paths
        t['class_name'] = t['image_url'].map(lambda s: s.split('/')[0])
        
        chunksize=100000
        startind = 0
        iter_csv = pd.read_csv(extracted_emb_file, iterator=True, header=None, chunksize=chunksize)
        for chunk in iter_csv:
            write_chunk = pd.DataFrame()
            endind = startind+chunksize
            if endind > len(extracted_img_paths):
                endind = len(extracted_img_paths)
            ind = np.arange(startind, endind)
            print(ind[0], ind[-1])
            startind = endind
            write_chunk = t.iloc[ind]
            write_chunk['feature'] = chunk[0].apply(lambda e: ",".join(map(str, e.split())))
            # print("write_chunk", write_chunk.head())
            with open(save_tsv_file, 'a') as f:
                write_chunk.to_csv(f, sep='\t', index=False, header=None)
                # pass
            # break
    return t

t = create_verified_tsv(type_name='all')


(0, 99999)


/home/krsharma/miniconda2/envs/env_repl/lib/python2.7/site-packages/ipykernel/__main__.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(100000, 199999)
(200000, 299999)
(300000, 310008)


In [200]:
# CLOTHING META FIX

# clean_test_kv (image_url, label) 10K
# noisy_train_kv (image_url, label) 1m
# 47569 clean_train_key_list.txt
# 14312 clean_val_key_list.txt

# verified_train.tsv 25K (image_url, verilabel)
# verified_val.tsv 7K (image_url, verilabel)

clean_val_key_list = '../datasets/clothing1m/anno/clean_val_key_list.txt'
clean_val = pd.read_csv(clean_val_key_list, sep=' ', header=None)
# print(clean_val.head())
# print(len(clean_val))


clean_train_key_list = '../datasets/clothing1m/anno/clean_train_key_list.txt'
clean_train = pd.read_csv(clean_train_key_list, sep=' ', header=None)
# print(clean_train.head())
# print(len(clean_train))


noisy_label_kv_list = '../datasets/clothing1m/anno/noisy_label_kv.txt'
noisy_label_kv = pd.read_csv(noisy_label_kv_list, sep=' ', header=None)
# print(noisy_label_kv.head())
# print(len(noisy_label_kv))

clean_label_kv_list = '../datasets/clothing1m/anno/clean_label_kv.txt'
clean_label_kv = pd.read_csv(clean_label_kv_list, sep=' ', header=None)
# print(clean_label_kv.head())
# print(len(clean_label_kv))

def veri_table(clean_tab, noisy_kv, clean_kv, name):
    v = noisy_kv[noisy_kv[0].isin(clean_tab[0])]
    v.columns = ['img_url', 'class_name']
    t = v.merge(clean_kv, on='img_url')
    t['verification_label'] = (t['class_name'] == t['gt_class'])*1
    print(t.head())
    print("num:", len(t))
    t.to_csv('../datasets/clothing1m/verified_{}.tsv'.format(name), index=False, sep='\t')
    return t
    
clean_label_kv.columns = ['img_url', 'gt_class']
verified_val = veri_table(clean_val, noisy_label_kv, clean_label_kv, 'val')
verified_train = veri_table(clean_train, noisy_label_kv, clean_label_kv, 'train')


In [224]:
verified_val = pd.read_csv('../datasets/clothing1m/verified_{}.tsv'.format('val'), sep='\t')
verified_train = pd.read_csv('../datasets/clothing1m/verified_{}.tsv'.format('train'), sep='\t')
# print(len(verified_val), verified_val.head())
# print(len(verified_train), verified_train.head())

extra_verified_df = pd.concat([verified_val, verified_train])[['img_url', 'class_name']]
# print(extra_verified_df.head())
# extra_verified_df.to_csv('../datasets/clothing1m/extra_verified_kv.tsv', index=False, header=None, sep=' ')

In [235]:
# CLOTHING
    

def create_verified_tsv(type_name):
    
    extracted_img_paths_file = "../trained_models/clothing1m_10e/extracted_imgpaths.txt"
    extracted_emb_file = '../trained_models/clothing1m_10e/extracted_emb.txt'  
    # class_name of 1st 1m are in noisy_train_v, rest are in verified df directly
    noisy_classlabels_file = '../datasets/clothing1m/noisy_train_kv.txt'
    
    if type_name == 'verified_train':
        v_list = "../datasets/clothing1m/verified_train.tsv"
        save_tsv_file = "../trained_models/clothing1m_10e/cleannet_train.tsv"
        vdf_colnames = ['image_url', 'class_name', 'gt_class', 'verification_label']
    elif type_name == 'verified_val':
        v_list = "../datasets/clothing1m/verified_val.tsv"
        save_tsv_file = "../trained_models/clothing1m_10e/cleannet_val.tsv"
        vdf_colnames = ['image_url', 'class_name', 'gt_class', 'verification_label']
    elif type_name == 'all_1m':
        save_tsv_file = "../trained_models/clothing1m_10e/cleannet_all_1m.tsv"
    elif type_name == 'all_veri':
        cleannet_val_file = '../trained_models/clothing1m_10e/cleannet_val.tsv'
        cleannet_train_file = '../trained_models/clothing1m_10e/cleannet_train.tsv'
        save_tsv_file = "../trained_models/clothing1m_10e/cleannet_all_veri.tsv"
        
    extracted_img_paths = pd.read_csv(extracted_img_paths_file, header=None, index_col=None, sep='\t')
    print("extracted_img_paths", len(extracted_img_paths), extracted_img_paths.head())

    if type_name in ['verified_train', 'verified_val']:
        vdf = pd.read_csv(v_list, '\t')
        vdf.columns = vdf_colnames
        print("verified df", vdf.head())
        
        ind = extracted_img_paths.index[extracted_img_paths[0].isin(vdf['image_url'])].tolist()
        print("ind", ind[0:10])
        
        # sample_key	image_url	class_name	verification_label	feature
        t = pd.DataFrame()
        t['sample_key'] = ind
        t['image_url'] = np.array(extracted_img_paths.iloc[ind])
        t = t.merge(vdf)
        # iter_csv = pd.read_csv(extracted_emb_file, iterator=True, header=None, chunksize=1000)
        # emb = pd.concat([chunk[chunk.index.isin(ind)] for chunk in iter_csv])
        a = np.arange(len(extracted_img_paths))
        set_ind = set(ind)
        skip = [i for i in a if i not in set_ind]
        print("skip", skip[0:10])
        emb_df = pd.read_csv(extracted_emb_file, skiprows=skip, header=None)
        t['feature'] = emb_df[0].apply(lambda e: ",".join(map(str, e.split())))
        t = t[['sample_key', 'image_url', 'class_name', 'verification_label', 'feature']]
        t.to_csv(save_tsv_file, sep='\t', index=False, header=None)
        print("tsv created", t.head())
    elif type_name=='all_1m':
        t = pd.DataFrame()
        t['sample_key'] = np.arange(len(extracted_img_paths))
        t['image_url'] = extracted_img_paths
        noisy_classlabels = pd.read_csv(noisy_classlabels_file, sep=' ', header=None)
        noisy_classlabels.columns = ['image_url', 'class_name']
        noisy_classlabels['class_name'].apply(lambda i: str(i))
        t['class_name'] = noisy_classlabels['class_name']  # already ordered
        # since we don't shuffle batches and read images from filelist into data loader of feat extractor  
        chunksize=100000
        startind = 0
        iter_csv = pd.read_csv(extracted_emb_file, iterator=True, header=None, chunksize=chunksize)
        for chunk in iter_csv:
            write_chunk = pd.DataFrame()
            endind = startind+chunksize
            if endind > len(extracted_img_paths):
                endind = len(extracted_img_paths)
            ind = np.arange(startind, endind)
            print(ind[0], ind[-1])
            startind = endind
            write_chunk = t.iloc[ind]
            write_chunk['feature'] = chunk[0].apply(lambda e: ",".join(map(str, e.split())))
            # print("write_chunk", write_chunk.head())
            with open(save_tsv_file, 'a') as f:
                write_chunk.to_csv(f, sep='\t', index=False, header=None)
                # pass
            # break
    elif type_name=='all_veri':
        t_val = pd.read_csv(cleannet_val_file, sep='\t', header=None)
        t_train = pd.read_csv(cleannet_train_file, sep='\t', header=None)
        write_colnames = ['sample_key', 'image_url', 'class_name', 'feature']
        t = pd.concat([t_val, t_train])
        t.columns = ['sample_key', 'image_url', 'class_name', 'verification_label', 'feature'] 
        t = t[write_colnames]
        t.to_csv(save_tsv_file, sep='\t', index=False, header=None)
        print("tsv created", len(t), t.head())
       
    return t

t = create_verified_tsv(type_name='all_veri')
# cat cleannet_all_1m.tsv cleannet_all_veri.tsv > cleannet_all.tsv
# similar to extracted files (imgpaths, emb) creation from subsets (noisy1m, verified)

('extracted_img_paths', 1032102,                                       0
0   images/2/46/50748838,3981129246.jpg
1  images/2/56/860908654,4267574256.jpg
2  images/9/38/436489045,3143453938.jpg
3  images/0/67/486092382,2324083067.jpg
4   images/4/55/4029117294,35432455.jpg)
('tsv created', 32102,    sample_key                              image_url  class_name  \
0     1000000  images/2/48/2334049070,2826935248.jpg           2   
1     1000001  images/4/84/2554049206,3987464484.jpg           6   
2     1000002  images/4/39/3843500707,1475408439.jpg          11   
3     1000003  images/4/99/3410098674,2546777499.jpg          11   
4     1000004   images/5/99/597790164,1391702599.jpg           9   

                                             feature  
0  0.081,0.149,0.103,0.532,0.259,0.422,0.805,0.59...  
1  0.000,0.162,0.056,0.397,0.805,1.394,0.605,0.35...  
2  0.201,0.164,0.976,0.534,0.556,0.312,0.733,0.02...  
3  0.262,0.261,0.457,0.752,0.532,0.628,0.128,0.21...  
4  0.386,0.263,0.81